In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriatic_Arthritis/GSE57376'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Synovial biopsies from RA and PsA patients and skin biopsies from Psoriasis patients under Infliximab treatment"
!Series_summary	"Object: to understand Infliximab treatment effect on the molecular expression of tissue at disease site"
!Series_overall_design	"4mm punch biopsies were performed on involved and uninvolved skin at baseline in 5 Ps patients. A repeat biopsy was performed at week 2 after IFX therapy at a site adjacent to the baseline biopsy of involved skin. Synovial biopsies were performed on the knee of 3 RA and 3 PsA paired-subjects with a Parker Pearson biopsy needle (Dyna Medical, London, Canada) under ultrasound guidance at baseline and repeated on the same knee at week 10"
Sample Characteristics Dictionary:
{0: ['Sex: F', 'Sex: M'], 1: ['age: 51', 'age: 28', 'age: 46', 'age: 57', 'age: 61', 'age: 35', 'age: 19', 'age: 67', 'age: 38', 'age: 55', 'age: 39', 'age: 44', 'age: 52'], 2: ['sample type: biopsy'], 3: ['tissue: knee', 'tiss

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data
is_gene_available = True

# Check availability and determine keys for variables
trait_row = 5 if set(['Psoriatic Arthritis']).intersection(set(['Psoriatic Arthritis', 'Psoriasis', 'Rheumatoid Arthritis'])) else None
age_row = 1 if len(set(['51', '28', '46', '57', '61', '35', '19', '67', '38', '55', '39', '44', '52'])) > 1 else None
gender_row = 0 if len(set(['F', 'M'])) > 1 else None

# Define conversion functions

def convert_trait(value):
    value = value.split(":")[1].strip() if ":" in value else value
    if value == "Psoriatic Arthritis":
        return 1
    elif value in ["Rheumatoid Arthritis", "Psoriasis"]:
        return 0
    return None

def convert_age(value):
    try:
        return float(value.split(":")[1].strip())
    except ValueError:
        return None
    except IndexError:
        return None
    
def convert_gender(value):
    value = value.split(":")[1].strip() if ":" in value else value
    if value == "F":
        return 0
    elif value == "M":
        return 1
    return None

# Save cohort information
save_cohort_info('GSE57376', './preprocessed/Psoriatic_Arthritis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriatic_Arthritis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriatic_Arthritis/trait_data/GSE57376.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1381406': [0.0, 51.0, 0.0], 'GSM1381407': [0.0, 28.0, 0.0], 'GSM1381408': [0.0, 46.0, 1.0], 'GSM1381409': [0.0, 57.0, 1.0], 'GSM1381410': [0.0, 61.0, 1.0], 'GSM1381411': [0.0, 35.0, 0.0], 'GSM1381412': [0.0, 28.0, 0.0], 'GSM1381413': [0.0, 19.0, 0.0], 'GSM1381414': [0.0, 28.0, 0.0], 'GSM1381415': [0.0, 61.0, 1.0], 'GSM1381416': [0.0, 57.0, 1.0], 'GSM1381417': [0.0, 35.0, 0.0], 'GSM1381418': [0.0, 19.0, 0.0], 'GSM1381419': [1.0, 67.0, 1.0], 'GSM1381420': [1.0, 38.0, 0.0], 'GSM1381422': [1.0, 55.0, 1.0], 'GSM1381423': [1.0, 39.0, 1.0], 'GSM1381424': [1.0, 55.0, 1.0], 'GSM1381425': [0.0, 19.0, 0.0], 'GSM1381426': [0.0, 61.0, 1.0], 'GSM1381427': [0.0, 28.0, 0.0], 'GSM1381428': [0.0, 35.0, 0.0], 'GSM1381429': [0.0, 57.0, 1.0], 'GSM1381430': [0.0, 51.0, 0.0], 'GSM1381431': [0.0, 28.0, 0.0], 'GSM1381432': [0.0, 28.0, 0.0], 'GSM1381433': [0.0, 46.0, 1.0], 'GSM1381434': [1.0, 44.0, 1.0], 'GSM1381435': [1.0, 67.0, 1.0], 'GSM1381436': [1.0, 52.0, 0.0], 'GSM1381437': [1.0, 39.0, 1.0], 'GSM138

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify keys for the identifier and gene symbol
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_dataframe = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping_dataframe)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Psoriatic_Arthritis/gene_data/GSE57376.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Psoriatic_Arthritis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE57376', './preprocessed/Psoriatic_Arthritis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Psoriatic_Arthritis/GSE57376.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Psoriatic_Arthritis', the least common label is '1.0' with 10 occurrences. This represents 31.25% of the dataset.
The distribution of the feature 'Psoriatic_Arthritis' in this dataset is fine.

Quartiles for 'Age':
  25%: 28.0
  50% (Median): 45.0
  75%: 55.5
Min: 19.0
Max: 67.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 16 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

